In [1]:
%cd C:\Users\lukec\PycharmProjects\emissions-tracking-conda

C:\Users\lukec\PycharmProjects\emissions-tracking-conda


In [27]:
# %%writefile functions/graph_utilities.py

import itertools
import numpy as np
import pandas as pd
from more_itertools import locate
import torch
from torch_geometric.data import Data

def concat_lists(lists):
    """List concatenation utility function"""
    return list(itertools.chain.from_iterable(lists))

def pd_to_adj_matrix(df:pd.DataFrame, columns:list, weights=False):
    """Form adjacency matrix from pandas dataframe columns"""
    adj, edgeWeights = np.zeros([2,0]), []
    for i, column in enumerate(columns):
        edgeDest = [list(locate(df[column], lambda x: x==i)) for i in df[column]]
        edgeOrig = concat_lists([list(np.ones(len(x)).astype(int)*i) for i, x in enumerate(edgeDest)])
        edgeDest = concat_lists(edgeDest)
        nonSelf = np.where([val!=edgeDest[i] for i, val in enumerate(edgeOrig)])[0]
        adj = np.concatenate((adj,np.array([np.take(edgeOrig,nonSelf), np.take(edgeDest,nonSelf)])),axis=-1)
        if weights: edgeWeights = edgeWeights + [weights[i]]*len(edgeDest)
    return adj.astype(int), edgeWeights

def build(df, edges:list, y_col=False, bins=False):
    """This function builds a graph given a Pandas DataFrame"""
    adjacency, _ = pd_to_adj_matrix(df, edges)
    edge_index = torch.tensor(adjacency, dtype=torch.long)
    data_x = df.drop(y_col, axis=1) if y_col else df
    x = torch.tensor(data.values, dtype=torch.float)
    if y_col:
        if bins:
            y = torch.tensor(np.digitize(data[y_col],bins=np.linspace(min(data[y_col]),max(data[y_col]),bins))-1, dtype=torch.long)
        else: y = torch.tensor(data[y_col], dtype=torch.float)
        return Data(x=x, edge_index=edge_index, y=y)
    else: return Data(x=x, edge_index=edge_index)

In [28]:
# %%writefile scripts/graph_run.py

import pandas as pd
# Import data
data = pd.read_csv("C:/Users\lukec\OneDrive - University of Cambridge\PhD\Data\Aggregate/US_2019.csv", low_memory=False)
graph = build(data, ['PRODUCT','SITE'],y_col='CO2e',bins=5)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [24]:
data['CO2e']

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
2430   NaN
2431   NaN
2432   NaN
2433   NaN
2434   NaN
Name: CO2e, Length: 2435, dtype: float64